In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [ ]:
sample_submission = pd.read_csv('tmp/data/raw/sample_submission.csv').set_index('ID')
print(sample_submission.shape)
sample_submission.head()

## Process Test

Let's start with our test set and work backwords so we don't engineer features unreleated to it.

To maintain the integrity of the output we need to generate from the test set, set we need to predict on, we replace the duplicate shop IDs rather than dropping them.

In [ ]:
test = pd.read_csv('tmp/data/raw/test.csv').set_index('ID')
test.loc[test['shop_id'] == 0, 'shop_id'] = 57  # Якутск Орджоникидзе, 56
test.loc[test['shop_id'] == 1, 'shop_id'] = 58  # Якутск ТЦ "Центральный"
test.loc[test['shop_id'] == 10, 'shop_id'] = 11  # Жуковский ул. Чкалова 39м²
test.to_pickle('tmp/data/processed/test.pkl')
print(test.shape)
test.head(5)

In [ ]:
# set some global variables for future gleaning
unique_test_shops = test['shop_id'].unique().tolist()
unique_test_items = test['item_id'].unique().tolist()
print(f'Unique shops:\t{len(unique_test_shops)}')
print(f'Unique items:\t{len(unique_test_items)}')

## Process Items

In [ ]:
#drop('item_name', axis=1)
items = pd.read_csv('tmp/data/raw/items.csv')
print(items.shape)
items.head()

In [ ]:
# drop items we won't predict on
# we could tokenize the item_names for similarity, but we won't in order to minimize complexity
items = items.loc[items['item_id'].isin(unique_test_items)].reset_index(drop=True)
print(items.shape)
items.to_pickle('tmp/data/processed/items.pkl')
items.head()

In [ ]:
unique_test_categories = items['item_category_id'].unique().tolist()
print(f'Unique categories:\t{len(unique_test_categories)}')

## Process Categories

In [ ]:
cats = pd.read_csv('tmp/data/raw/item_categories.csv')
print(cats.shape)
cats.head()

In [ ]:
# drop items we won't predict on
cats = cats.loc[cats['item_category_id'].isin(unique_test_categories)].reset_index(drop=True)
print(cats.shape)
cats.head()

Honestly, I manually googled the Russian text, and saw that the categories had inconsisent naming conventions. Rather than writing explicit rules to clean the text, I decided to try a naive approach by tokenizing the categories and 1-hot encoding them. This way, a category that had `PC` at the start or end of a category would still get the `PC` feature.

In [ ]:
# Categories could be general enough for help indicate popularity/sales, so we translate them to features.
cats['split'] = cats['item_category_name'].apply(lambda x: x.split('-'))
#cats['cat'] = cats['split'].apply(lambda x: x[0].strip())
#cats['subcat'] = cats['split'].apply(lambda x: x[1].strip() if len(x) > 1 else np.NaN)
cats['split'] = cats['item_category_name'].apply(lambda x: x.replace('- ', ' ').replace(':', ' ').split())
cats['split'] = cats['split'].apply(lambda x: [i.strip().lstrip('(').rstrip(')').strip() for i in x])

binarizer = MultiLabelBinarizer().fit(cats['split'])
cat_classes = [f'cat_{c}' for c in binarizer.classes_]
cats[cat_classes] = binarizer.transform(cats['split'])
cats = cats.drop(['item_category_name', 'split'], axis=1)
feats = cats[cats.columns[1:]].sum().reset_index().sort_values(0)
feats = ['item_category_id'] + feats.loc[feats[0]>1, 'index'].tolist()
cats = cats[feats]

cats.head()

In [ ]:
items = items.drop('item_name', axis=1)
cats = items.merge(cats, on='item_category_id', how='left')
cats = cats.drop('item_category_id', axis=1)
cats.to_pickle('tmp/data/processed/categories.pkl')
print(cats.shape)
cats.head()

## Process Shops

Filter for the shops only in the test set, which inherently drops the mislabeled shops.

In [ ]:
shops = pd.read_csv('tmp/data/raw/shops.csv')
shops = shops.loc[shops['shop_id'].isin(unique_test_shops)]
print(shops.shape)
shops.head()

Candidly, I googled a lot of the Russian words to discover the shop names contain cities, shop types, and addresses, so we create binarized features cities and shop types.

In [ ]:
# City features

# Выездная Торговля = offsite trade
# Интернет-магазин ЧС = emergency online store
# Цифровой склад 1С-Онлайн = digital warehouse 1-c online
other = list(['Выездная Торговля', 'Интернет-магазин ЧС', 'Цифровой склад 1С-Онлайн'])

shops['city'] = np.NaN
shops.loc[~shops['shop_name'].isin(other), 'city'] = shops.loc[~shops['shop_name'].isin(other), 'shop_name'].apply(lambda x: x.split(' ')[0])
shops.loc[shops['city']=='Сергиев', 'city'] = 'Сергиев Посад'
shops.head()

In [ ]:
# Shop Type features
traveling = 'Выездная Торговля'
online = list(['Интернет-магазин ЧС', 'Цифровой склад 1С-Онлайн'])

shops['shop_type'] = np.NaN # np.NaN
shop_type = ['ТЦ', 'ТРК', 'ТРЦ', 'МТРЦ', 'ТК']
for st in shop_type:
    shops.loc[shops['shop_name'].str.contains(st), 'shop_type'] = st

shops.loc[shops['shop_name'].isin(online), 'shop_type'] = 'Online'
shops.loc[shops['shop_name']==traveling, 'shop_type'] = 'Traveling'

shops.head()

In [ ]:
# Address was considered as a feature, but we will avoid it to reduce complexity
shops['address'] = np.NaN # np.NaN
for i in shops.index:
    shop_name = shops.loc[i, 'shop_name']
    city = shops.loc[i, 'city']
    shop_type = shops.loc[i, 'shop_type']
    
    address = shop_name
    for n in (other + list([city]) + list([shop_type])):
        if isinstance(n, str):
            address = address.lstrip(n).strip()
    shops.loc[i, 'address'] = address
    
shops.head()

In [ ]:
binarizer = LabelBinarizer().fit(shops['city'].fillna('None'))
city_classes = [f'city_{c}' for c in [c.replace(' ', '_') for c in binarizer.classes_]]
shops[city_classes] = binarizer.transform(shops['city'].fillna('None'))

binarizer = LabelBinarizer().fit(shops['shop_type'].fillna('None'))
shop_type_classes = [f'shop_type_{c}' for c in [c.replace(' ', '_') for c in binarizer.classes_]]
shops[shop_type_classes] = binarizer.transform(shops['shop_type'].fillna('None'))
    
shops = shops.drop(['shop_name', 'address', 'city', 'city_None', 'shop_type', 'shop_type_None'], axis=1)
shops = shops.fillna(0)
shops.to_pickle('tmp/data/processed/shops.pkl')
print(shops.shape)
shops.head()

## Process Training Data

In [ ]:
train = pd.read_csv('tmp/data/raw/sales_train.csv')
print(train.shape)
train.head()

In [ ]:
train = train.loc[train['shop_id'].isin(unique_test_shops) & train['item_id'].isin(unique_test_items)]
print(train.shape)
train.head()

In [ ]:
# convert dates
train['date'] = pd.to_datetime(train['date'], format="%d.%m.%Y")
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train.drop(['date'], axis=1, inplace=True)

train['revenue'] = train['item_price'] * train['item_cnt_day']

train = train.groupby(['date_block_num', 'month', 'year', 'shop_id', 'item_id']).agg({'revenue':'sum', 'item_cnt_day':'sum'}).reset_index()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})

# Kaggle competition states output is capped at 20, so we clip our outputs to predict on
train['item_cnt_month'] = train['item_cnt_month'].clip(0, 20)

print(train.shape)
train.describe()

In [ ]:
# Rebuild the dataframe with timestamps and zero sale months
cols = ['date_block_num', 'month', 'year', 'shop_id', 'item_id']
dates = train.copy().groupby(['date_block_num', 'month', 'year']).sum().reset_index()[['date_block_num', 'month', 'year']]

train_mod = list()
for dtup in dates.itertuples():
    for shop_id in unique_test_shops:
        for item_id in unique_test_items:
            vals = list([dtup[1], dtup[2], dtup[3], shop_id, item_id])
            train_mod.append(vals)
            
train_mod = pd.DataFrame(train_mod, columns=cols)

# merge expanded dataset
train = train_mod.merge(train, on=cols, how='left').fillna(0)
del train_mod
train.to_pickle('tmp/data/processed/train.pkl')
print(train.shape)
train.head(5)

## Revisit the test set to add features

In [ ]:
test = pd.read_pickle('tmp/data/processed/test.pkl')
print(test.shape)
test.head()

In [ ]:
train.loc[train['date_block_num']==33].groupby(['date_block_num', 'month', 'year']).count().reset_index()[['date_block_num', 'month', 'year']]

In [ ]:
test['date_block_num'] = 34
test['month'] = 11
test['year'] = 2015
test['revenue'] = 0
test['item_cnt_month'] = 0
test = test[['date_block_num', 'month', 'year', 'shop_id', 'item_id', 'revenue', 'item_cnt_month']]
test.to_pickle('tmp/data/processed/test_plus_features.pkl')
test.head(5)